# 1. Introduction

Describe the problem the software solves and why it's important to solve that problem.


- 

# 2. Background 

Describe (briefly) the mathematical background and concepts as you see fit. You do not need to give a treatise on automatic differentiation or dual numbers. Just give the essential ideas (e.g. the chain rule, the graph structure of calculations, elementary functions, etc). Do not copy and paste any of the lecture notes. We will easily be able to tell if you did this as it does not show that you truly understand the problem at hand. 



- 

# 3. How to Use

How do you envision that a user will interact with your package? What should they import? How can they instantiate AD objects?

Note: This section should be a mix of pseudo code and text. It should not include any actual operations yet. Remember, you have not yet written any code at this point. 

In [1]:
# Codes 


# 4. Software Organization 

Discuss how you plan on organizing your software package.

What will the directory structure look like?
What modules do you plan on including? What is their basic functionality?
Where will your test suite live? Will you use TravisCI? CodeCov?
How will you distribute your package (e.g. PyPI)?
How will you package your software? Will you use a framework? If so, which one and why? If not, why not?
Other considerations?


# 5. Implementation

Discuss how you plan on implementing the forward mode of automatic differentiation.

What are the core data structures?
What classes will you implement?
What method and name attributes will your classes have?
What external dependencies will you rely on?
How will you deal with elementary functions like sin, sqrt, log, and exp (and all the others)?
Be sure to consider a variety of use cases. For example, don't limit your design to scalar functions of scalar values. Make sure you can handle the situations of vector functions of vectors and scalar functions of vectors. Don't forget that people will want to use your library in algorithms like Newton's method (among others).


- 

# 6. Licensing 

Licensing is an essential consideration when you create new software. You should choose a suitable license for your project. A comprehensive list of licenses can be found here. The license you choose depends on factors such as what other software or libraries do you use in your code (copyleft, copyright), will you have to deal with patents, how can others advertise software that makes use of your code (or parts thereof)? You may consult the following reading to aid you in choosing a license:

Helper to choose a license
Licenses
License recommendations
License compatibility
Extensive list of open source licenses
Briefly motivate your license choice in the milesone1 document.

- 